In [4]:
import json
import pandas as pd
import numpy as np

In [5]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return int(x)  # Convert to integer if the float is a whole number
        else:
            return '{:.2f}'.format(x).rstrip('0').rstrip('.')  # Keep up to 2 decimals, remove trailing zeros
    return x

def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
            for layout, metrics in benchmark_value.items():
                if layout == 'init':
                    continue
                # if 'lookahead_swap' not in metrics:
                #     continue
                basic_swap = metrics.get('basic_swap', np.nan)
                sabre_swap = metrics.get('sabre_swap', np.nan)
                lookahead_swap = metrics.get('lookahead_swap', np.nan)
# Percentage Reduction=( Swap Sabre−Swap Lookahead) / Swap Sabre * 100
                basic_swap_reduction = ((basic_swap - lookahead_swap) / basic_swap * 100) if basic_swap != 0 and basic_swap is not np.nan and lookahead_swap is not np.nan else np.nan
                sabre_swap_reduction = ((sabre_swap - lookahead_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not np.nan and lookahead_swap is not np.nan else np.nan
                
                basic_depth = metrics.get('basic_depth', np.nan)
                sabre_depth = metrics.get('sabre_depth', np.nan)
                lookahead_depth = metrics.get('lookahead_depth', np.nan)
                basic_depth_reduction = ((basic_depth - lookahead_depth) / basic_depth * 100) if basic_depth != 0 and basic_depth is not np.nan and lookahead_depth is not np.nan else np.nan
                sabre_depth_reduction = ((sabre_depth - lookahead_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not np.nan and lookahead_depth is not np.nan else np.nan

                data.append({
                    "benchmark": benchmark,
                    "size": circuit, # circuit size
                    "gate": benchmark_value['init']['size'], # total gates
                    "depth": benchmark_value['init']['depth'], # circuit depth
                    "layout": layout,
                    "swap basic": basic_swap, # basic swap
                    "swap sabre": sabre_swap, # sabre swap
                    "swap lookahead": lookahead_swap, # lookahead swap
                    "swap delta basic": basic_swap_reduction,
                    "swap delta sabre": sabre_swap_reduction, # difference between sabre and lookahead, if positive then reduced but if negative then larger than sabre
                    "depth basic": basic_depth, # basic depth
                    "depth sabre": sabre_depth, # sabre depth
                    "depth lookahead": lookahead_depth, # lookahead depth
                    "depth delta basic": basic_depth_reduction,
                    "depth delta sabre": sabre_depth_reduction
                })
    return pd.DataFrame(data)


In [6]:
def rearrange_columns(df, first_col_name):
    """
    Reorder the columns of the DataFrame so that the specified column is first.
    
    Parameters:
    - df: The input DataFrame.
    - first_col_name: The name of the column to move to the first position.
    
    Returns:
    - A DataFrame with the specified column moved to the first position.
    """
    # Check if the specified column exists in the DataFrame
    if first_col_name not in df.columns:
        raise ValueError(f"Column '{first_col_name}' does not exist in the DataFrame.")
    
    # Create a list of columns with the specified column first
    columns = [first_col_name] + [col for col in df.columns if col != first_col_name]
    
    # Reorder the DataFrame
    df_reordered = df[columns]
    
    return df_reordered

In [7]:
def open_json_from_file():
    with open("result/benchmarking_FINAL.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()
print(len(dict_benchmark["5"].keys()))

15


$\Delta s_B$

In [8]:
df = flatten_json(dict_benchmark)
df = df.map(format_number)

def rename_columns(df):
    df.columns = [
        "benchmark",
        # "n",
        "g",
        "d",
        "layout",
        r"$s_B$", # basic swap
        r"$s_S$", # sabre swap
        r"$s_L$", # lookahead swap
        # "swap (%)",
        r"$\Delta s_B$", # swap basic reduction
        r"$\Delta s_S$", # swap sabre reduction
        r"$d_B$", # basic depth
        r"$d_S$", # sabre depth
        r"$d_L$", # lookahead depth
        # "d (%)",
        r"$\Delta d_B$",
        r"$\Delta d_S$"
    ]
    return df
# df = rename_columns(df)

print(df)

    benchmark size  gate  depth            layout  swap basic  swap sabre  \
0          dj    5    36     11         full_20_1           0           0   
1          dj    5    36     11         line_20_1          36           6   
2          dj    5    36     11         full_10_2           0           3   
3          dj    5    36     11          full_7_3           0           0   
4          dj    5    36     11          grid_9_2           9           3   
..        ...  ...   ...    ...               ...         ...         ...   
445    wstate   15   253    135          grid_8_3          21           3   
446    wstate   15   253    135         ring_10_2           0          42   
447    wstate   15   253    135          ring_7_3           0          27   
448    wstate   15   253    135  t_horizontal_5_4          63           3   
449    wstate   15   253    135    t_vertical_5_4         126          15   

    swap lookahead swap delta basic swap delta sabre  depth basic  \
0     

In [9]:
def save_latex_to_file(latex, circuit_size, filename=''):
    # with open(f"table/table_{filename}_{circuit_size}.tex", "w") as file:
    with open(f"table/data_{filename}_{circuit_size}_FINAL.tex", "w") as file:
        file.write(latex)

In [10]:
def build_by_benchmark(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    # df_n_5 = df_n_5.replace('_', r'\_', regex=True)
    df = df.replace('_', '\\_', regex=True)
    df = df.sort_values(by=['gate', 'benchmark'])
    df = df.drop(columns=['size'])
    df = rename_columns(df)
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    # latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'benchmark')
    return df

In [11]:
import os

def build_by_benchmark_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.sort_values(by=['gate', 'benchmark'])
    # df = df.drop(columns=['size'])
    # df = rename_columns(df)
    # with pd.ExcelWriter("excel/benchmark_layout.xlsx", mode="a" if os.path.exists("excel/benchmark_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
    with pd.ExcelWriter("excel/data_benchmark_4.xlsx", mode="a" if os.path.exists("excel/data_benchmark_4.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [12]:
df_benchmark_5 = build_by_benchmark(df, 5)
df_benchmark_10 = build_by_benchmark(df, 10)
df_benchmark_15 = build_by_benchmark(df, 15)

/tmp/ipykernel_38582/461077298.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('_', '\\_', regex=True)


In [10]:
build_by_benchmark_excel(df, 15)
build_by_benchmark_excel(df, 10)
build_by_benchmark_excel(df, 5)

,benchmark,size,gate,depth,layout,swap basic,swap sabre,swap lookahead,swap delta basic,swap delta sabre,depth basic,depth sabre,depth lookahead,depth delta basic,depth delta sabre
93,ghz,15,17,17,grid_6_4,15,30,39,-160,-30,32,44,29,9.38,34.09
94,ghz,15,17,17,ring_5_4,0,18,72,nan,-300,17,35,34,-100,2.86
95,ghz,15,17,17,full_5_4,0,18,nan,nan,nan,17,32,nan,nan,nan
90,dj,15,118,22,full_5_4,114,27,9,92.11,66.67,101,61,32,68.32,47.54
91,dj,15,118,22,grid_6_4,261,36,21,91.95,41.67,125,77,40,68,48.05
92,dj,15,118,22,ring_5_4,153,36,30,80.39,16.67,113,71,32,71.68,54.93
96,graphstate,15,150,30,grid_6_4,87,24,54,37.93,-125,88,34,28,68.18,17.65
97,graphstate,15,150,30,ring_5_4,93,27,78,16.13,-188.89,92,36,36,60.87,0
98,graphstate,15,150,30,full_5_4,63,12,36,42.86,-200,73,36,38,47.95,-5.56
129,vqe,15,253,31,full_5_4,0,15,nan,nan,nan,31,50,nan,nan,nan


In [13]:
def build_by_layout(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    df = df.replace('_', '\\_', regex=True)
    df = df.drop(columns=['size']) # remove circuit size
    df = rename_columns(df)
    df = rearrange_columns(df, 'layout')
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    # latex = latex.replace('_', r'\_').replace('%', r'\%')
    save_latex_to_file(latex, circuit_size, 'layout')
    return df

In [15]:
def build_by_layout_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    # df = df.drop(columns=['size']) # remove circuit size
    # df = rename_columns(df)
    df = rearrange_columns(df, 'layout')

    # with pd.ExcelWriter("excel/excel_layout.xlsx", mode="a" if os.path.exists("excel/benchmark_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
    with pd.ExcelWriter("excel/data_layout_4.xlsx", mode="a" if os.path.exists("excel/data_layout_4.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [16]:
df_layout_5 = build_by_layout(df, 5)
df_layout_10 = build_by_layout(df, 10)
df_layout_15 = build_by_layout(df, 15)

/tmp/ipykernel_38582/2456750932.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_38582/2456750932.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('_', '\\_', regex=True)
/tmp/ipykernel_38582/2456750932.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas

In [14]:
build_by_layout_excel(df, 15)
build_by_layout_excel(df, 10)
build_by_layout_excel(df, 5)

/tmp/ipykernel_79284/1521635181.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_79284/1521635181.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_79284/1521635181.py:3: DeprecationWarning: DataFram

layout       benchmark size  gate  depth  swap basic  \
layout                                                                 
full_5_4 95   full_5_4             ghz   15    17     17           0   
         90   full_5_4              dj   15   118     22         114   
         98   full_5_4      graphstate   15   150     30          63   
         129  full_5_4             vqe   15   253     31           0   
         132  full_5_4          wstate   15   253    135           0   
         105  full_5_4            qaoa   15   285     34         135   
         108  full_5_4             qft   15   591    118         735   
         111  full_5_4    qftentangled   15   608    122         735   
         122  full_5_4   realamprandom   15   615     77        2385   
         128  full_5_4  twolocalrandom   15   615     77        2385   
         123  full_5_4       su2random   15   675     81        2385   
         114  full_5_4             qnn   15   914    158        1548   
         101  full_5_4   portfolioqaoa   15  1260    192        2385   
         117  full_5_4          random   15  1992    412        1023   
         102  full_5_4    portfoliovqe   15  2505    327        2385   
grid_6_4 93   grid_6_4             ghz   15    17     17          15   
         91   grid_6_4              dj   15   118     22         261   
         96   grid_6_4      graphstate   15   150     30          87   
         130  grid_6_4             vqe   15   253     31          66   
         133  grid_6_4          wstate   15   253    135          21   
         106  grid_6_4            qaoa   15   285     34         300   
         109  grid_6_4             qft   15   591    118        1584   
         112  grid_6_4    qftentangled   15   608    122        1425   
         120  grid_6_4   realamprandom   15   615     77        4404   
         126  grid_6_4  twolocalrandom   15   615     77        4404   
         124  grid_6_4       su2random   15   675     81        4404   
         115  grid_6_4             qnn   15   914    158        2721   
         99   grid_6_4   portfolioqaoa   15  1260    192        4404   
         118  grid_6_4          random   15  1992    412        1821   
         103  grid_6_4    portfoliovqe   15  2505    327        4404   
ring_5_4 94   ring_5_4             ghz   15    17     17           0   
         92   ring_5_4              dj   15   118     22         153   
         97   ring_5_4      graphstate   15   150     30          93   
         131  ring_5_4             vqe   15   253     31           0   
         134  ring_5_4          wstate   15   253    135           0   
         107  ring_5_4            qaoa   15   285     34         171   
         110  ring_5_4             qft   15   591    118        1089   
         113  ring_5_4    qftentangled   15   608    122        1089   
         121  ring_5_4   realamprandom   15   615     77        2934   
         127  ring_5_4  twolocalrandom   15   615     77        2934   
         125  ring_5_4       su2random   15   675     81        2934   
         116  ring_5_4             qnn   15   914    158        2007   
         100  ring_5_4   portfolioqaoa   15  1260    192        2934   
         119  ring_5_4          random   15  1992    412        1872   
         104  ring_5_4    portfoliovqe   15  2505    327        2934   

              swap sabre swap lookahead swap delta basic swap delta sabre  \
layout                                                                      
full_5_4 95           18            nan              nan              nan   
         90           27              9            92.11            66.67   
         98           12             36            42.86             -200   
         129          15            nan              nan              nan   
         132          15            nan              nan              nan   
         105          21             63            53.33             -200   
         108   

In [15]:
print("Benchmark [5]\n", df_benchmark_5.value_counts("benchmark"))
print("Benchmark [10]\n", df_benchmark_10.value_counts("benchmark"))
print("Benchmark [15]\n", df_benchmark_15.value_counts("benchmark"))

Benchmark [5]
 benchmark
dj                3
ghz               3
graphstate        3
portfolioqaoa     3
portfoliovqe      3
qaoa              3
qft               3
qftentangled      3
qnn               3
random            3
realamprandom     3
su2random         3
twolocalrandom    3
vqe               3
wstate            3
Name: count, dtype: int64
Benchmark [10]
 benchmark
dj                3
ghz               3
graphstate        3
portfolioqaoa     3
portfoliovqe      3
qaoa              3
qft               3
qftentangled      3
qnn               3
random            3
realamprandom     3
su2random         3
twolocalrandom    3
vqe               3
wstate            3
Name: count, dtype: int64
Benchmark [15]
 benchmark
dj                3
ghz               3
graphstate        3
portfolioqaoa     3
portfoliovqe      3
qaoa              3
qft               3
qftentangled      3
qnn               3
random            3
realamprandom     3
su2random         3
twolocalrandom    3
vqe        

In [16]:
df_layout_15

layout       benchmark     g    d  $s_B$  $s_S$ $s_L$  \
layout                                                                    
full_5_4 95   full\_5\_4             ghz    17   17      0     18   nan   
         90   full\_5\_4              dj   118   22    114     27     9   
         98   full\_5\_4      graphstate   150   30     63     12    36   
         129  full\_5\_4             vqe   253   31      0     15   nan   
         132  full\_5\_4          wstate   253  135      0     15   nan   
         105  full\_5\_4            qaoa   285   34    135     21    63   
         108  full\_5\_4             qft   591  118    735    162   nan   
         111  full\_5\_4    qftentangled   608  122    735    195   nan   
         122  full\_5\_4   realamprandom   615   77   2385    600   nan   
         128  full\_5\_4  twolocalrandom   615   77   2385    579   nan   
         123  full\_5\_4       su2random   675   81   2385    654   nan   
         114  full\_5\_4             qnn   914  158   1548    417   nan   
         101  full\_5\_4   portfolioqaoa  1260  192   2385    555   nan   
         117  full\_5\_4          random  1992  412   1023    711   nan   
         102  full\_5\_4    portfoliovqe  2505  327   2385    624   nan   
grid_6_4 93   grid\_6\_4             ghz    17   17     15     30    39   
         91   grid\_6\_4              dj   118   22    261     36    21   
         96   grid\_6\_4      graphstate   150   30     87     24    54   
         130  grid\_6\_4             vqe   253   31     66      3    54   
         133  grid\_6\_4          wstate   253  135     21     21    39   
         106  grid\_6\_4            qaoa   285   34    300     48   135   
         109  grid\_6\_4             qft   591  118   1584    303   441   
         112  grid\_6\_4    qftentangled   608  122   1425    297   537   
         120  grid\_6\_4   realamprandom   615   77   4404    648   711   
         126  grid\_6\_4  twolocalrandom   615   77   4404    645   711   
         124  grid\_6\_4       su2random   675   81   4404    618   711   
         115  grid\_6\_4             qnn   914  158   2721    450   813   
         99   grid\_6\_4   portfolioqaoa  1260  192   4404    597  1197   
         118  grid\_6\_4          random  1992  412   1821    924  1257   
         103  grid\_6\_4    portfoliovqe  2505  327   4404    684   744   
ring_5_4 94   ring\_5\_4             ghz    17   17      0     18    72   
         92   ring\_5\_4              dj   118   22    153     36    30   
         97   ring\_5\_4      graphstate   150   30     93     27    78   
         131  ring\_5\_4             vqe   253   31      0     18   nan   
         134  ring\_5\_4          wstate   253  135      0     30   123   
         107  ring\_5\_4            qaoa   285   34    171     54   102   
         110  ring\_5\_4             qft   591  118   1089    309   nan   
         113  ring\_5\_4    qftentangled   608  122   1089    336   nan   
         121  ring\_5\_4   realamprandom   615   77   2934    888   nan   
         127  ring\_5\_4  twolocalrandom   615   77   2934    777   nan   
         125  ring\_5\_4       su2random   675   81   2934    831   nan   
         116  ring\_5\_4             qnn   914  158   2007    528   nan   
         100  ring\_5\_4   portfolioqaoa  1260  192   2934    765   nan   
         119  ring\_5\_4          random  1992  412   1872    927   nan   
         104  ring\_5\_4    portfoliovqe  2505  327   2934    900   nan   

             $\Delta s_B$ $\Delta s_S$  $d_B$  $d_S$ $d_L$ $\Delta d_B$  \
layout                                                                    
full_5_4 95           nan          nan     17     32   nan          nan   
         90         92.11        66.67    101     61    32        68.32   
         98         42.86         -200     73     36    38        47.95   
         129          nan          nan     31     50   nan          nan   
         132          nan          nan    